Assignment 12

In [178]:
import numpy as np
import cv2

class Cave:
    def __init__(self, name):
        self.name = name
        # if name is in capitals, it's a big cave
        self.big = name.isupper()
        self.Connected = []

    def AddTunnel(self, cave):
        self.Connected.append(cave)
        self.Connected=sorted(self.Connected, key=lambda x: x.name)

    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name

    def __eq__(self, __o: object) -> bool:
        return self.name == __o.name
        
    # print all connected caves
    def PrintConnected(self):
        print(self.Connected)
    



# Add to caves 
def AddToCaves(caves, cave):
    if cave not in caves:
        #print("Adding " + cave.name)
        caves.append(cave)
        return caves, cave
    else:
        for j in range(0, len(caves)):
            if caves[j].name == cave.name:
                #print("Already added " + caves[j].name)
                return caves, caves[j]    
                



# Get the area of the image of from the document
def GetCaveSystem(dataset):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    data = [x.strip() for x in data]
    data = [x.split('-') for x in data]

    # create the cave system
    caves = []
    for i in range(len(data)):
        # add the cave to the cave system
        caves, tmpCave1 = AddToCaves(caves, Cave(data[i][0]))
        caves, tmpCave2 = AddToCaves(caves, Cave(data[i][1]))
        # add the tunnels
        tmpCave1.AddTunnel(tmpCave2)
        tmpCave2.AddTunnel(tmpCave1)

    caves = sorted(caves, key=lambda x: x.name)
    return caves

# breath first traverse the cave system and return all the paths. 
# If the cave is small only travese it one time. 
# If its a big cave, It can travese it multiple times. 
# do this until all path combination are found and return the paths.
def TraverseCaveSystem(cave, visited, end):
    paths = []
    if cave.name == end.name:
        paths.append([cave])
        return paths
    for i in range(0, len(cave.Connected)):
        if cave.Connected[i] not in visited or cave.Connected[i].big:
            newVisited = visited + [cave.Connected[i]]
            tmp = TraverseCaveSystem(cave.Connected[i], newVisited, end)
            for j in range(0, len(tmp)):
                paths.append([cave] + tmp[j])

    return paths

def FindStartAndEnd(caves):
    for i in range(0, len(caves)):
        if caves[i].name == 'start':
            start = caves[i]
        if caves[i].name == 'end':
            end = caves[i]
    return start, end
            


In [179]:
data = GetCaveSystem('./data/aoc12.txt')

start , end = FindStartAndEnd(data)
paths = TraverseCaveSystem(start, [start], end)

print(len(paths))


5212


Part 2

In [180]:
def TraverseCaveSystemNew(cave, visited, end, small = [[]]):
    paths = []
    if cave.name == end.name:
        paths.append([cave])
        return paths
    for i in range(0, len(cave.Connected)):
        if cave.Connected[i].big:
            newVisited = visited + [cave.Connected[i]]
            tmp = TraverseCaveSystemNew(cave.Connected[i], newVisited, end, small)
            for j in range(0, len(tmp)):
                paths.append([cave] + tmp[j])
            continue

        elif cave.Connected[i] not in visited:
            newVisited = visited + [cave.Connected[i]]
            newSmall = small + [[cave.Connected[i],1]]
            tmp = TraverseCaveSystemNew(cave.Connected[i], newVisited, end, newSmall)
            for j in range(0, len(tmp)):
                paths.append([cave] + tmp[j])
            continue
        
        if cave.Connected[i].name == 'start':
            continue

        else:
            twice = False
            for j in range(0, len(small)):
                if small[j][1] == 2:
                    twice = True
                    break
            if not twice:
                newVisited = visited + [cave.Connected[i]]
                newSmall = small + [[cave.Connected[i],2]]
                tmp = TraverseCaveSystemNew(cave.Connected[i], newVisited, end, newSmall)
                for j in range(0, len(tmp)):
                    paths.append([cave] + tmp[j])

    return paths
    

In [181]:

data = GetCaveSystem('./data/aoc12.txt')

# find the start and end cave
start , end = FindStartAndEnd(data)
paths = TraverseCaveSystemNew(start, [start], end, [[start,1]])
print(len(paths))
#print(paths)

134862
